In [25]:
import requests
import pandas as pd

In [2]:
# Esta funcion imprime las 10 monedas volatiles con mas capitalizacion y luego las 10 monedas estables.
def get_top_coins():
    stable = []
    volatil = []

    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()

    for coin in data:
        if coin['current_price'] > 1 or 0.99 > coin['current_price'] and len(volatil) < 10:
            volatil.append(coin['id'])
        elif 1.1 > coin['current_price'] > 0.97 and len(stable) < 10:
            stable.append(coin['id'])
        else:
            break
        
    return stable,volatil

stable, volatil = get_top_coins()
print(stable)
print(volatil)

['tether', 'usd-coin']
['bitcoin', 'ethereum', 'binancecoin', 'ripple', 'staked-ether', 'cardano', 'dogecoin', 'solana', 'tron', 'polkadot']


In [185]:
# Utiliza una lista de nombres de criptomonedas y a traves de la api de coingecko extrae la informacion de cada una de ellas para luego de un ETL guardar 
# la informacion en un archivo csv.
def get_historical_data(coinnamelist,days):
    for coin in coinnamelist:
        url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart"
        params = {
            'vs_currency': 'usd',
            'days': days,
            'interval': 'daily'
        }
    
        response = requests.get(url, params=params)
    
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)
            df['market_caps'] = df['market_caps'].apply(lambda x: int(x[1]))
            df['total_volumes'] = df['total_volumes'].apply(lambda x: int(x[1]))
            df['Date'] = df['prices'].apply(lambda x: pd.to_datetime(x[0], unit='ms'))
            df['prices'] = df['prices'].apply(lambda x: int(x[1]))
            df.rename(columns={'prices': 'Price_usd','total_volumes':'Volume','market_caps':'Market_cap'}, inplace=True)
            df = df.groupby(df["Date"].dt.to_period("M")).tail(1)
            df.to_csv(f'./Data/{coin}.csv',index=False)
        else:
            print(f"Error al obtener los datos de {coin}:", response.status_code)


In [28]:
get_historical_data(volatil)

***

Obtencion de datos globales

In [3]:
url = 'https://api.coingecko.com/api/v3/global'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

In [4]:
data

{'data': {'active_cryptocurrencies': 10150,
  'upcoming_icos': 0,
  'ongoing_icos': 49,
  'ended_icos': 3376,
  'markets': 836,
  'total_market_cap': {'btc': 41909168.0563866,
   'eth': 659824072.8019023,
   'ltc': 16787531520.292736,
   'bch': 5725551987.714669,
   'bnb': 5107265646.735155,
   'eos': 1869087785530.507,
   'xrp': 2084736384445.723,
   'xlm': 8719990169319.429,
   'link': 174022950370.4369,
   'dot': 247451527525.28854,
   'yfi': 196650428.46162176,
   'usd': 1111275380341.9402,
   'aed': 4081753366634.2646,
   'ars': 388900709701547.44,
   'aud': 1715246881905.506,
   'bdt': 121618100976189.36,
   'bhd': 418990824302.6044,
   'bmd': 1111275380341.9402,
   'brl': 5400909475999.873,
   'cad': 1502434312743.8823,
   'chf': 975814245304.3989,
   'clp': 958819510912830.6,
   'cny': 7986736158517.532,
   'czk': 24635820567060.652,
   'dkk': 7622962385313.375,
   'eur': 1022662281513.4741,
   'gbp': 872999047115.1628,
   'hkd': 8713454693492.135,
   'huf': 391243091509044.3,


In [5]:
data = data['data']
df_global = pd.DataFrame(data)
df_global.head()

,active_cryptocurrencies,upcoming_icos,ongoing_icos,ended_icos,markets,total_market_cap,total_volume,market_cap_percentage,market_cap_change_percentage_24h_usd,updated_at
btc,10150,0,49,3376,836,4.190917e+07,1.946119e+06,46.447861,2.646764,1692812860
eth,10150,0,49,3376,836,6.598241e+08,3.063998e+07,18.243934,2.646764,1692812860
ltc,10150,0,49,3376,836,1.678753e+10,7.795558e+08,NaN,2.646764,1692812860
bch,10150,0,49,3376,836,5.725552e+09,2.658751e+08,NaN,2.646764,1692812860
bnb,10150,0,49,3376,836,5.107266e+09,2.371640e+08,3.017118,2.646764,1692812860


In [150]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, btc to sol
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   active_cryptocurrencies               67 non-null     int64  
 1   upcoming_icos                         67 non-null     int64  
 2   ongoing_icos                          67 non-null     int64  
 3   ended_icos                            67 non-null     int64  
 4   markets                               67 non-null     int64  
 5   total_market_cap                      61 non-null     float64
 6   total_volume                          61 non-null     float64
 7   market_cap_percentage                 10 non-null     float64
 8   market_cap_change_percentage_24h_usd  67 non-null     float64
 9   updated_at                            67 non-null     int64  
dtypes: float64(4), int64(6)
memory usage: 5.8+ KB


Faltan datos importante en las columnas total_market_cap y market_cap_percentage

In [6]:
df_global['market_cap_percentage'].sum()

83.74989519178826

La sumatoria de la columna market_cap_percentage (porcentaje de capitalizacion) no llega al 100 % o cercano.

In [12]:
df_global = df_global.sort_values(by='market_cap_percentage', ascending=False)
df_global.head()

AttributeError: 'list' object has no attribute 'sort_values'

***

Utilizando la opcion /coins/market de la api de coingecko para obtener datos mas completos.

In [14]:
# Esta funcion retorna los datos de todas las criptomonedas en coingecko
def all_coins():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 250,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()
        
    return data

In [15]:
df_global = all_coins()

In [16]:
df_global = pd.DataFrame(df_global)
df_global.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,26098.000000,508115315953,1,5.481153e+11,6.401115e+09,26391.000000,...,2.100000e+07,2.100000e+07,69045.00,-62.18853,2021-11-10T14:24:11.849Z,67.810000,38400.56100,2013-07-06T00:00:00.000Z,None,2023-08-25T12:59:52.454Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1654.220000,198890570225,2,1.988906e+11,6.788454e+09,1667.550000,...,1.202164e+08,NaN,4878.26,-66.06976,2021-11-10T14:24:19.604Z,0.432979,382183.21615,2015-10-20T00:00:00.000Z,"{'times': 83.74456319474335, 'currency': 'btc'...",2023-08-25T12:59:45.512Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999797,82831846090,3,8.283185e+10,1.646600e+10,1.003000,...,8.284913e+10,NaN,1.32,-24.41547,2018-07-24T00:00:00.000Z,0.572521,74.67576,2015-03-02T00:00:00.000Z,None,2023-08-25T12:55:00.330Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,216.170000,33289170677,4,4.327311e+10,3.929972e+08,219.420000,...,1.538562e+08,2.000000e+08,686.31,-68.47108,2021-05-10T07:24:17.097Z,0.039818,543339.40584,2017-10-19T00:00:00.000Z,None,2023-08-25T12:59:44.270Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.514163,27214550827,5,5.143148e+10,8.178154e+08,0.523685,...,9.998849e+10,1.000000e+11,3.40,-84.85765,2018-01-07T00:00:00.000Z,0.002686,19057.31709,2014-05-22T00:00:00.000Z,None,2023-08-25T12:59:49.955Z


In [17]:
df_global.columns

Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated'],
      dtype='object')

Comprobando la esactitud calculando el porcentaje de capitalizacion o dominancia de bitcoin con los datos provistos.

In [18]:
df_global[df_global['id'] == 'bitcoin']['market_cap'][0]/df_global['market_cap'].sum()

0.47569763162872286

Segun https://coin360.com/ los datos son correctos.

Eliminando columnas innecesarias

In [19]:
df_global.drop(['image','high_24h','ath_date','atl_date','low_24h','price_change_24h','price_change_percentage_24h','market_cap_change_24h','market_cap_change_percentage_24h','roi'],axis=1,inplace=True)

In [37]:
df_global.head()

,id,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,atl,atl_change_percentage,last_updated
0,bitcoin,btc,Bitcoin,26098.000000,508115315953,1,5.481153e+11,6.401115e+09,1.946748e+07,2.100000e+07,2.100000e+07,69045.00,-62.18853,67.810000,38400.56100,2023-08-25T12:59:52.454Z
1,ethereum,eth,Ethereum,1654.220000,198890570225,2,1.988906e+11,6.788454e+09,1.202164e+08,1.202164e+08,NaN,4878.26,-66.06976,0.432979,382183.21615,2023-08-25T12:59:45.512Z
2,tether,usdt,Tether,0.999797,82831846090,3,8.283185e+10,1.646600e+10,8.284913e+10,8.284913e+10,NaN,1.32,-24.41547,0.572521,74.67576,2023-08-25T12:55:00.330Z
3,binancecoin,bnb,BNB,216.170000,33289170677,4,4.327311e+10,3.929972e+08,1.538562e+08,1.538562e+08,2.000000e+08,686.31,-68.47108,0.039818,543339.40584,2023-08-25T12:59:44.270Z
4,ripple,xrp,XRP,0.514163,27214550827,5,5.143148e+10,8.178154e+08,5.291419e+10,9.998849e+10,1.000000e+11,3.40,-84.85765,0.002686,19057.31709,2023-08-25T12:59:49.955Z


In [38]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       250 non-null    object 
 1   symbol                   250 non-null    object 
 2   name                     250 non-null    object 
 3   current_price            250 non-null    float64
 4   market_cap               250 non-null    int64  
 5   market_cap_rank          250 non-null    int64  
 6   fully_diluted_valuation  239 non-null    float64
 7   total_volume             250 non-null    float64
 8   circulating_supply       250 non-null    float64
 9   total_supply             240 non-null    float64
 10  max_supply               146 non-null    float64
 11  ath                      250 non-null    float64
 12  ath_change_percentage    250 non-null    float64
 13  atl                      250 non-null    float64
 14  atl_change_percentage    2

El dataframe no contiene datos nulos o vacios.

In [39]:
df_global.to_csv('./Data/all_coins.csv',index=False)

***

Extrayendo informacion de exchanges

In [40]:
def get_exchange_info(limit=None, base_currency=None):
    base_url = "https://api.coingecko.com/api/v3/exchanges"
    params = {'per_page': 250}
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        exchanges_data = response.json()
        return exchanges_data
    else:
        print("Error:", response.status_code)
        return None

In [41]:
df_exchanges = get_exchange_info()

In [42]:
df_exchanges = pd.DataFrame(df_exchanges)
df_exchanges

,id,name,year_established,country,description,url,image,has_trading_incentive,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,binance,Binance,2017.0,Cayman Islands,,https://www.binance.com/,https://assets.coingecko.com/markets/images/52...,False,10.0,1.0,200152.154441,131061.926907
1,gdax,Coinbase Exchange,2012.0,United States,,https://coinbase-consumer.sjv.io/coingecko,https://assets.coingecko.com/markets/images/23...,False,10.0,2.0,29557.805294,29557.805294
2,bybit_spot,Bybit,2018.0,British Virgin Islands,Bybit is a cryptocurrency exchange that offers...,https://www.bybit.com,https://assets.coingecko.com/markets/images/69...,False,10.0,3.0,28586.516671,28586.516671
3,huobi,Huobi,2013.0,Seychelles,,https://www.huobi.com,https://assets.coingecko.com/markets/images/25...,False,10.0,4.0,38743.010827,19735.354488
4,kraken,Kraken,2011.0,United States,,https://r.kraken.com/c/2223866/687155/10583,https://assets.coingecko.com/markets/images/29...,False,10.0,5.0,18165.400205,13077.545471
...,...,...,...,...,...,...,...,...,...,...,...,...
245,fusionx-v2,FusionX V2,NaN,None,,https://fusionx.finance/swap?,https://assets.coingecko.com/markets/images/11...,False,5.0,245.0,0.272881,0.272881
246,sushiswap_xdai,Sushiswap (xDai),NaN,None,Sushiswap on xDai is a DeFi protocol providing...,https://app.sushi.com/swap,https://assets.coingecko.com/markets/images/67...,False,5.0,246.0,0.269402,0.269402
247,dystopia,Dystopia,NaN,None,,https://www.dystopia.exchange/,https://assets.coingecko.com/markets/images/88...,False,5.0,247.0,0.257524,0.257524
248,baryon_network,Baryon Network,NaN,None,,https://www.baryon.network/swap,https://assets.coingecko.com/markets/images/92...,False,5.0,248.0,0.181071,0.181071


In [43]:
df_exchanges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               250 non-null    object 
 1   name                             250 non-null    object 
 2   year_established                 140 non-null    float64
 3   country                          129 non-null    object 
 4   description                      245 non-null    object 
 5   url                              250 non-null    object 
 6   image                            250 non-null    object 
 7   has_trading_incentive            247 non-null    object 
 8   trust_score                      249 non-null    float64
 9   trust_score_rank                 249 non-null    float64
 10  trade_volume_24h_btc             250 non-null    float64
 11  trade_volume_24h_btc_normalized  250 non-null    float64
dtypes: float64(5), object(

In [44]:
df_exchanges['has_trading_incentive'].unique()

array([False, None], dtype=object)

In [45]:
# Elimina las columnas con datos no relevantes
df_exchanges.drop(['url','image','description','has_trading_incentive'],axis=1,inplace=True)

In [46]:
df_exchanges[['trade_volume_24h_btc','trade_volume_24h_btc_normalized']] = df_exchanges[['trade_volume_24h_btc','trade_volume_24h_btc_normalized']].round(2)

In [53]:
df_exchanges[['year_established','trust_score_rank','trust_score']] = df_exchanges[['year_established','trust_score_rank','trust_score']].astype('Int64')

In [54]:
df_exchanges.head()

,id,name,year_established,country,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,binance,Binance,2017,Cayman Islands,10,1,200152.15,131061.93
1,gdax,Coinbase Exchange,2012,United States,10,2,29557.81,29557.81
2,bybit_spot,Bybit,2018,British Virgin Islands,10,3,28586.52,28586.52
3,huobi,Huobi,2013,Seychelles,10,4,38743.01,19735.35
4,kraken,Kraken,2011,United States,10,5,18165.40,13077.55


In [36]:
df_exchanges.to_csv('./Data/exchanges.csv',index=False)